In [1]:
from config import *
from db import *
from utilities import *

In [3]:
from time import sleep
import json
from pathlib import Path

In [4]:
import pymongo 

client = pymongo.MongoClient(f"mongodb+srv://{mongo_user}:{mongo_password}@{mongo_url}")
db = client.walmart

In [5]:
try:
    current_path = os.path.dirname(os.path.abspath(__file__))
except:
    current_path = '.'

In [6]:
driver = init_driver(gecko_driver, user_agent = user_agent, is_headless = False)

In [7]:
is_login = load_cookies(driver)

if is_login == False:
    twitter_login(driver)

In [8]:
driver.get(twitter_url)

In [ ]:
if len driver.find_elements_by_css_selector('public-DraftStyleDefault-block') 
tweet_box = driver.find_elements_by_css_selector('public-DraftStyleDefault-block')
if len(tweet)